In [2]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 84.8 MB/s eta 0:00:00


In [3]:
# FAISS 사용
import faiss
import numpy as np

In [4]:
# 데이터 생성
np.random.seed(42)
d = 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')  # 데이터베이스
xq = np.random.randn(5,d).astype('float32')  # 쿼리  5개

# 정확한 검색
index_flat = faiss.IndexFlatL2(d)
index_flat.add(xb)

k = 4 # 상위 4개 검색
D, I  = index_flat.search(xq, k)
print('Flat 인덱스 결과')
print(f'거리:{D}')
print(f'인덱스:{I}')

Flat 인덱스 결과
거리:[[176.22398 179.27386 181.42618 183.5282 ]
 [158.36673 159.89926 169.92009 172.46124]
 [141.05345 144.73184 150.72574 152.20578]
 [156.71101 165.06566 172.04468 177.16544]
 [142.94458 160.52213 163.8829  168.60358]]
인덱스:[[2689 3260  321 3219]
 [ 195 2470 2498 8653]
 [5837 3019 5030 2459]
 [6238 5120 4717 4445]
 [9679 2947 9567  684]]


IVF (Inverted File Index)

원리: 벡터 공간을 여러 클러스터로 분할하고, 검색 시 관련 클러스터만 탐색

In [6]:
d = 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')  # 데이터베이스
# IVF 인덱스 생성
nlist = 100 # 클러스터 수
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer,d,nlist)
#학습(클러스터링)
index.train(xb)
# 데이터 추가
index.add(xb)
# 검색(nprobe 조절)
index.nprobe = 10  # 클러스터 수
D,I =  index.search(xq, k=2)
print('IVF 인덱스 결과')
print(f'거리:{D}')
print(f'인덱스:{I}')

IVF 인덱스 결과
거리:[[176.37585 177.63406]
 [165.18779 166.15088]
 [144.07776 154.57266]
 [175.07066 177.26479]
 [167.02963 169.66336]]
인덱스:[[8777 5695]
 [6124 5716]
 [3054 3110]
 [ 530  736]
 [2544 4868]]


Product Quantization (PQ)  

원리: 고차원 벡터를 여러 서브벡터로 나누고 각각을 양자화

In [8]:
d = 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')  # 데이터베이스

# IVF + PQ 인덱스
nlist = 100 # 클러스터 수
m = 8 # 서브벡터 수( d가 m으로 나눠떨어져야 함)
nbits = 8  # 각 서브벡터당 비트 수( 2^8) = 256 클러스터

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer,d,nlist,m,nbits)
# 학습
index.train(xb)
#추가
index.add(xb)
#검색
index.nprobe = 10
D,I =  index.search(xq, k=2)
# 메모리 비교
flat_memory = n*d*4
pq_memory = n*m  # 각 벡터당 m바이트
print(f'Flat 메모리 : {flat_memory / 1e6:.2f}MB')
print(f'PQ 메모리 : {pq_memory / 1e6:.2f}MB')
print(f'압축률 : {flat_memory / pq_memory:.0f}MB')




Flat 메모리 : 5.12MB
PQ 메모리 : 0.08MB
압축률 : 64MB


HNSW (Hierarchical Navigable Small World)

원리: 다층 그래프 구조로 효율적인 탐색

In [10]:
d = 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')  # 데이터베이스
# HNSW 인덱스
M=32 # 각 노드의 연결 수
index = faiss.IndexHNSWFlat(d,M)
# 인덱스 구축시 탐색 깊이
index.hnsw.efConstruction = 40
# 데이터 추가(학습불필요)
index.add(xb)
# 검색시 탐색 깊이
index.hnsw.efSearch = 16
D,I = index.search(xq, k=5)

In [ ]:
import faiss
import numpy as np
import time

def benchmark_index(index, xb, xq, name):
    """인덱스 성능 벤치마크"""
    # 추가 시간
    start = time.time()
    if hasattr(index, 'train'):
        index.train(xb)
    index.add(xb)
    add_time = time.time() - start

    # 검색 시간
    start = time.time()
    for _ in range(10):
        index.search(xq, k=10)
    search_time = (time.time() - start) / 10

    print(f"{name}:")
    print(f"  추가 시간: {add_time:.3f}s")
    print(f"  검색 시간: {search_time*1000:.2f}ms")

# 데이터 준비
d, n = 128, 100000
xb = np.random.randn(n, d).astype('float32')
xq = np.random.randn(100, d).astype('float32')

# Flat
benchmark_index(faiss.IndexFlatL2(d), xb.copy(), xq, "Flat")

# IVF
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)
benchmark_index(index_ivf, xb.copy(), xq, "IVF")

# HNSW
benchmark_index(faiss.IndexHNSWFlat(d, 32), xb.copy(), xq, "HNSW")

Flat:
  추가 시간: 0.038s
  검색 시간: 67.40ms
IVF:
  추가 시간: 0.289s
  검색 시간: 3.41ms


< 1만: IndexFlatL2 (정확도 우선)

1만 ~ 100만: IndexIVFFlat (균형)

100만: IndexIVFPQ (메모리 효율)